# `StableSpectralElements.jl` - 1D linear advection-diffusion example

We first need to load StableSpectralElements.jl and OrdinaryDiffEq, which handle the spatial and temporal components of the discretization, respectively. The packages on the second line are used in this example for visualization and displaying timing information.

In [1]:
using StableSpectralElements, OrdinaryDiffEq
using Plots, Plots.PlotMeasures, Printf, TimerOutputs

[ Info: Precompiling StableSpectralElements [fb992021-99c7-4c2d-a14b-5e48ac4045b2]


We will solve the one-dimensional linear advection-diffusion equation
$$
\partial_t U(x,t) + \partial_x (a U(x,t) - b \partial_x U(x,t)) = 0,  \quad \forall\, (x, t) \in (0,L) \times (0,T),
$$
with $a = 1$, $b = 5 \times 10^{-2}$, $L = 1$, and $T = 1$, imposing periodic boundary conditions as well as the initial condition
$$
U(x,0) = \sin(2\pi x/L), \quad \forall \, x \in (0,L).
$$

In [2]:
a = 1.0  # wave speed
b = 5.0e-2  # diffusion coefficient
L = 1.0  # domain length
T = 1.0  # end time for one period

conservation_law = LinearAdvectionDiffusionEquation(a,b)
initial_data = InitialDataSine(1.0,2π/L)
exact_solution = ExactSolution(conservation_law,initial_data);

The problem described above will be discretized using a standard weak-form DG method of `NodalTensor` type on a uniform mesh with $M = 4$ elements of degree $p = 4$. The default options `inviscid_numerical_flux=LaxFriedrichsNumericalFlux()` and `viscous_numerical_flux=BR1()` are used.

In [3]:
M = 4
p = 4
form = StandardForm()

reference_approximation = ReferenceApproximation(
    NodalTensor(p), Line())

mesh = uniform_periodic_mesh(reference_approximation, (0.0,L), M)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation);

We'll now save the project. This sets up a directory where intermediate solution files are saved, and allows the simulation to be resumed from a previous state.

In [4]:
results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T),
     "results/advection_diffusion_1d/", clear=true, overwrite=true);

The function `semidiscretize` sets up the solver using the above parameters and uses it to construct an `ODEProblem` object. We can then use `solve` from OrdinaryDiffEq.jl to integrate in time. The five-stage, fourth-order low-storage explicit Runge-Kutta method from Carpenter and Kennedy is used for such a purpose. We use `save_callback` to write the solution to file 50 times during the course of the simulation.

In [5]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T), 
    PhysicalOperator())

CFL = 0.1
h = L / (reference_approximation.N_p * spatial_discretization.N_e)
dt = CFL * h / a

reset_timer!()
sol = solve(ode_problem, CarpenterKennedy2N54(), adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, (0.0,T),  
        floor(Int, T/(dt*50))))
print_timer()

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching +(::Nothing, ::Float64)
    
    [0mClosest candidates are:
    [0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
    [0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:578[24m[39m
    [0m  +([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64}
    [0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:408[24m[39m
    [0m  +([91m::ForwardDiff.Dual{Tx}[39m, ::AbstractFloat) where Tx
    [0m[90m   @[39m [35mForwardDiff[39m [90m~/.julia/packages/ForwardDiff/vXysl/src/[39m[90m[4mdual.jl:144[24m[39m
    [0m  ...
    
    Stacktrace:
     [1] [0m[1m_broadcast_getindex_evalf[22m
    [90m   @[39m [90m./[39m[90m[4mbroadcast.jl:683[24m[39m[90m [inlined][39m
     [2] [0m[1m_broadcast_getindex[22m
    [90m   @[39m [90m./[39m[90m[4mbroadcast.jl:656[24m[39m[90m [inlined][39m
     [3] [0m[1mgetindex[22m
    [90m   @[39m [90m./[39m[

The $L^2$ norm of the error in the numerical solution can now be approximated, where we use an over-integrated quadrature rule in this case.

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization, LGQuadrature(p+2))
println("L2 error:")
println(analyze(error_analysis, last(sol.u), exact_solution, T)...)

L2 error:


LoadError: UndefVarError: `sol` not defined

Using the snapshots taken during the simulation, the discrete conservation and energy functionals can be evaluated, corresponding to the discrete approximations of $\int_0^L U(x,t) \, \mathrm{d} x$ and $\tfrac{1}{2}\int_0^L U(x,t)^2 \, \mathrm{d} x$, respectively. Note that the conservation functional remains constant up to machine precision, while the energy functional is nonincreasing for all time.

In [7]:
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
p1 = plot(analyze(conservation_analysis, load_time_steps(results_path)), 
    ylabel="Conservation functional")
p2 = plot(analyze(energy_analysis, load_time_steps(results_path)), 
    ylabel="Energy functional");
plot(p1,p2, size=(600,300),margin=5mm, fmt=:png)

LoadError: BoundsError: attempt to access 0-element Vector{Float64} at index [1]

We can plot the numerical solution at time $t = T$ and compare it to the analytical solution.

In [8]:
plot(spatial_discretization, last(sol.u), 
    exact_solution=exact_solution, time=T, ylims=[-1.0,1.0], fmt=:png)

LoadError: UndefVarError: `sol` not defined

Such plots can then be used to create an animation as shown below. We'll save it as a `.gif` file.

In [9]:
anim = @animate for i ∈ eachindex(sol.u)
    plot(spatial_discretization, sol.u[i], 
        ylims=[-1.0,1.0], ylabel="\$U^h(x,t)\$",
        label=string("\$t = \$", @sprintf "\$%.2f\$" sol.t[i]))
end
gif(anim, "advection_diffusion_solution.gif", fps = 10)

LoadError: UndefVarError: `sol` not defined